In [1]:
import numpy as np
import pickle
from tqdm import tqdm
from gzhlaker import *

In [2]:
ph_memmap_train_path = "/sda/data/guozihang/memmap/phoenix2014-bigarray-map-train"
ph_memmap_train_info_path = "/sda/data/guozihang/memmap/phoenix2014-train.pickle"

In [3]:
pht_memmap_train_path = "/sda/data/guozihang/sign_dataset/pht_bigarray/phoenix2014-T-bigarray-map-train"
pht_memmap_train_info_path = "/sda/data/guozihang/sign_dataset/pht_bigarray/phoenix2014-T-train.pickle"

In [4]:
with open(ph_memmap_train_info_path, mode="rb") as f:
    ph_train_info = pickle.load(f)

In [5]:
with open(pht_memmap_train_info_path, mode="rb") as f:
    pht_train_info = pickle.load(f)

In [6]:
T, H, W, C = ph_train_info[-1]["end"], 256, 256, 3

In [7]:
ph_fp = np.memmap(ph_memmap_train_path, dtype='uint8', mode='r', shape=(T, H, W, C))

In [8]:
T, H, W, C = pht_train_info[-1]["end"], 256, 256, 3

In [9]:
pht_fp = np.memmap(pht_memmap_train_path, dtype='uint8', mode='r', shape=(T, H, W, C))

In [10]:
len(ph_train_info)

5671

In [11]:
ph_train_info[-1]

{'path': '/sda/data/guozihang/phoenix2014-release_cache/train/26May_2010_Wednesday_tagesschau_default-5.npy',
 'start': 798805,
 'end': 798990}

In [12]:
len(pht_train_info)

7096

In [13]:
for item in tqdm(pht_train_info):
    item['start'] += 798990
    item['end'] += 798990

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7096/7096 [00:00<00:00, 1443323.85it/s]


In [14]:
fuse_train_info = ph_train_info + pht_train_info

In [15]:
fuse_train_info[-1]

{'path': '/sda/data/guozihang/PHOENIX-2014-T-release-v3_cache/train/30November_2011_Wednesday_tagesschau-6285.npy',
 'start': 1626206,
 'end': 1626344}

In [17]:
T

1626344

In [16]:
T, H, W, C = fuse_train_info[-1]["end"], 256, 256, 3

In [18]:
fu_fp = np.memmap("/sda/data/guozihang/fuse_ph_memmap", dtype='uint8', mode='w+', shape=(T, H, W, C))

In [19]:
T

1626344

In [ ]:
for index, item in tqdm(enumerate(fuse_train_info)):
    path, start, end = item["path"], item["start"], item["end"]
    if index < 5671:
        fu_fp[start:end] = ph_fp[start:end]
    else:
        fu_fp[start:end] = pht_fp[start - 798990:end - 798990]
    if index % 500 == 0:
        fu_fp.flush()

41it [01:39,  2.69s/it]

In [ ]:
fu_fp.flush()